# 基于MindSpore实现AlexNet手写字体识别

本实验主要介绍使用华为的深度学习框架MindSpore实现的AlexNet模型，用于手写字体的图像分类任务，使用的数据集为MNIST手写数字数据集。该实验旨在展示如何使用MindSpore框架构建深度学习模型，并在手写数字分类任务中进行训练和测试。

## 1.实验目的

- 掌握如何使用MindSpore框架构建深度学习模型。

- 掌握如何在手写数字分类任务中使用MindSpore进行数据处理、模型构建、训练和评估。

- 了解AlexNet模型的基本结构和工作原理。

- 通过手写数字识别任务掌握深度学习在计算机视觉中的应用。

- 掌握如何使用深度学习模型解决实际问题。

## 2.AlexNet模型原理介绍

[AlexNet](http://www.cs.toronto.edu/~hinton/absps/imagenet.pdf)是一种深度卷积神经网络模型，是由Alex Krizhevsky、Ilya Sutskever和Geoffrey Hinton在2012年ImageNet大规模视觉识别竞赛（ImageNet Large Scale Visual Recognition Challenge, ILSVRC）中提出并夺得冠军的。AlexNet模型的主要结构包括5个卷积层和3个全连接层，其中，前5个卷积层之间以及最后2个全连接层之间使用了ReLU激活函数，而最后一个全连接层使用了softmax激活函数。

![jupyter](./Figures/Fig002.png)

[AlexNet](http://www.cs.toronto.edu/~hinton/absps/imagenet.pdf)的卷积层采用了较大的卷积核（11x11、5x5），较大的步幅（4、2），以及较多的卷积通道（96、256），这样的设计使得模型具有更强的表征能力和更高的分类准确度。同时，在模型训练时，[AlexNet](http://www.cs.toronto.edu/~hinton/absps/imagenet.pdf)采用了数据增强和dropout等技术，以避免模型出现过拟合现象，提高模型的泛化能力。



## 3.实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.4；Python环境=3.11。


|  硬件平台 |  操作系统  |          软件环境           | 开发环境 | 环境搭建链接 |
| :-----:| :----: |:-----------------------:|:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU |Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4.数据处理

### 4.1数据准备

MNIST数据集是机器学习中常用的数据集之一，它包含了70,000张手写数字图片，每张图片都是28x28像素大小的灰度图像，数字从0到9。该数据集由美国国家标准与技术研究所（NIST）于1999年创建，而后由Yann LeCun等人进行改进和维护，因此被称为MNIST（Modified NIST）数据集。

MNIST数据集主要用于图像识别领域的研究，特别是用于机器学习算法的测试和比较。由于它的简单性和易于使用，MNIST已经成为机器学习社区中的标准数据集之一。

以下是几个示例图像：

![jupyter](./Figures/Fig001.png)

每个图像都被转换成784个数字的一维向量（28×28=784）。这些数字表示像素的灰度值，值的范围从0（黑色）到255（白色）。为了将这些向量转换成有意义的输出，通常使用分类算法，例如支持向量机（SVM）或神经网络。分类算法的目标是将每个图像正确地标记为它所代表的数字。


### 4.2数据加载

MindSpore提供了MNIST数据集并将其划分为训练集和测试集，我们只需下载载入即可。
我们可以通过MnistDataset函数载入下载好的数据集，并且get_col_names()提供了查看数据集中所有列名称的功能。
载入完成数据后，我们需要对数据集进行预处理，我们通过datapipe()函数实现对图片的预处理。


In [1]:
import mindspore 
from mindspore import nn as nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset


#下载数据集
from download import download    

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 31.9MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [2]:
# 载入数据集
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

# 获取数据集中所有列的名称
train_dataset.get_col_names() 

['image', 'label']

In [3]:

# datapipe完成映射图像变换和将大规模数据集分割成多个小批次的数据集
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Resize((32, 32)),
        vision.HWC2CHW()
        
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


# 数据集预处理
train_dataset = datapipe(train_dataset, 16)
test_dataset = datapipe(test_dataset, 16)

In [4]:
# 结果展示 每个image包含16张图片，每张图片是单通道，高32像素，宽32像素
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


Shape of image [N, C, H, W]: (16, 1, 32, 32) Float32
Shape of label: (16,) Int32
Shape of image [N, C, H, W]: (16, 1, 32, 32) Float32
Shape of label: (16,) Int32


## 5.模型构建

构建一个减少卷积层的精简版AlexNet：第一层为conv1，采用relu激活函数，第二层为最大池化层，第三层为conv2，采用relu激活函数，第四层为最大池化层，第五层为全连接层，第六层为全连接层，第七层为全连接层。使用交叉熵损失函数，和Adam优化器。

In [5]:
'''
AlexNet的原始输入图片大小为224*224
Mnist数据集中图片大小为28*28
所以需要对网络进行精简，减少两层卷积层。
'''

class AlexNet(nn.Cell):
    def __init__(self, num_class=10, num_channel=1):
        super(AlexNet, self).__init__()
        # 卷积层，输入的通道数为num_channel，输出的通道数为6，卷积核大小为5*5
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        # 卷积层，输入的通道数为6，输出的通道数为16，卷积核大小为5*5
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        # 全连接层，输入个数为16*5*5，输出个数为120
        self.fc1 = nn.Dense(16 * 5 * 5, 120)
        # 全连接层，输入个数为120，输出个数为84
        self.fc2 = nn.Dense(120, 84)
        # 全连接层，输入个数为84，分类的个数为num_class
        self.fc3 = nn.Dense(84, num_class)
        # ReLU激活函数
        self.relu = nn.ReLU()
        # 池化层
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        # 多维数组展平为一维数组
        self.flatten = nn.Flatten()

    def construct(self, x):
        # 使用定义好的运算构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = AlexNet()

In [6]:
# 损失函数、优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Adam(params=model.trainable_params())

## 6.训练模型

此部分分为两个部分，训练和测试。测试部分不需要计算梯度通过model.set_train(False)实现。

In [7]:
def train(model, dataset, loss_fn, optimizer):
    # 定义前向传播
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # 获取梯度
    grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # 获取每个batch损失
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        optimizer(grads)
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
#训练3个周期
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    #每个周期训练完的模型在测试集上验证
    test(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308531  [  0/3750]
loss: 0.696949  [100/3750]
loss: 0.527091  [200/3750]
loss: 0.148770  [300/3750]
loss: 0.069595  [400/3750]
loss: 0.044726  [500/3750]
loss: 0.010204  [600/3750]
loss: 0.245963  [700/3750]
loss: 0.106098  [800/3750]
loss: 0.047226  [900/3750]
loss: 0.066341  [1000/3750]
loss: 0.117556  [1100/3750]
loss: 0.156105  [1200/3750]
loss: 0.029339  [1300/3750]
loss: 0.171240  [1400/3750]
loss: 0.009431  [1500/3750]
loss: 0.168849  [1600/3750]
loss: 0.030581  [1700/3750]
loss: 0.040762  [1800/3750]
loss: 0.081207  [1900/3750]
loss: 0.074026  [2000/3750]
loss: 0.288456  [2100/3750]
loss: 0.377680  [2200/3750]
loss: 0.009328  [2300/3750]
loss: 0.358196  [2400/3750]
loss: 0.022618  [2500/3750]
loss: 0.004836  [2600/3750]
loss: 0.160606  [2700/3750]
loss: 0.129163  [2800/3750]
loss: 0.446368  [2900/3750]
loss: 0.064398  [3000/3750]
loss: 0.020420  [3100/3750]
loss: 0.047853  [3200/3750]
loss: 0.035052  [3300/3750]
loss: 0.060732  [3

## 7.模型预测

MindSpore提供save_checkpoint()函数保存模型和load_checkpoint()函数载入模型，我们载入刚才训练好的模型，对测试集再次进行预测。

In [9]:

# 模型保存
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


In [10]:

# 实例化模型
model = AlexNet()
# 载入训练好的模型
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)

([], [])


In [11]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break



Predicted: "[2 2 8 1 9 3 6 6 0 4]", Actual: "[2 2 8 1 9 3 6 6 0 4]"
